# DICE Analysis Tutorial:
In this tutorial, we'll go over how to run the dice_correlation.py script. The purpose of the script is to allow for the calculation of the Dice Coefficient between ROIs of two different atlases. This script outputs a series of png and csv files containing the Dice Coefficients.

First the relevant libraries must be imported. dice_correlation utilizes the nibabel, numpy, argpars, matplotlib, os, sys, and math libraries.

In [1]:
import nibabel as nb
import numpy as np
from argparse import ArgumentParser
import matplotlib
from matplotlib import pyplot as plt
import os
import sys
from math import floor

Next we have to specify the inputs. For the purpose of this jupyter notebook sys.argv is used to store the inputs, but when running the script in the terminal:

python $<$path_to_script$>$/dice_correlation.py $<$input_dir$>$ $<$output_dir$>$ $<$atlas1$>$ $<$atlas2$>$ $<$atlas3$>$ ...
    
input_dir = The path to the dirctory containing the atlases you intend to analyze

output_dir = The path for the directory where you intend to have the output files saved
    
atlases = All the atlases you intend to have analyzed, seperated by spaces. You must provide at least 2 atlases, with no upper limit. Atlases should be of the same voxel size.

In [2]:
input_dir = '/Users/ross/Documents/neuroparc/atlases/label/Human'
output_dir = '/Users/ross/Documents/neuroparc/atlases'
atlases = ['AAL_space-MNI152NLin6_res-1x1x1.nii.gz','AICHAJoliot2015_space-MNI152NLin6_res-1x1x1.nii.gz']

#Necessary for running this function in a jupyter notebook
sys.argv = ['',input_dir, output_dir, atlases[0],atlases[1]]

Now the dice_roi function needs to be defined, which is where the majority of the calculations are done. The Dice Coefficients are calculated by itterating through every combination of ROIs from both atlases and storing them in an ndarray. That array is then used to create a png heatmap and a csv (comma delimited) file of the results.

If you wish to change the format of the png files, you are able to change any of the formating code after the comment "Generate png of heatmap" without compromising any of the calculations.

In [3]:
def dice_roi(input_dir, output_dir, atlas1, atlas2):
    """Calculates the dice coefficient for every ROI combination from atlas1 and atlas2

    Parameters
    ----------
    input_dir : str
        Path to input directory
    output_dir : str
        Path to output directory
    atlas1 : str
        path to first atlas to compare
    atlas2 : str
        path to second atlas to compare
    """

    #Create output name for png file
    yname = atlas1.split('_space-')[0]
    res=atlas1.split('space-MNI152NLin6_res-')[1]
    res=res.split('.nii')[0]
    xname = atlas2.split('_space-')[0]
    
    #Create name for the generate files
    png_name=f"DICE_{yname}_&_{xname}_res-{res}"

    at1 = nb.load(f'{input_dir}/{atlas1}')
    at2 = nb.load(f'{input_dir}/{atlas2}')

    atlas1 = at1.get_data()
    atlas2 = at2.get_data()
    
    #Get ROI numerical values for both atlases
    labs1 = np.unique(atlas1)
    labs2 = np.unique(atlas2)
    
    #Create ndarray of zeros to contain Dice Coefficients
    Dice = np.zeros((labs1.size, labs2.size))

    max_y=len(labs1)-1
    max_x=len(labs2)-1

    #Itterate through the ROIs of each atlas and calculate the Dice Coefficient
    for i in range(len(labs1)):
        val1=labs1[i]
        for j in range(len(labs2)):
            val2=labs2[j]
            #Calculate Dice Coefficient
            dice = np.sum(atlas1[atlas2==val2]==val1)*2.0 / (np.sum(atlas1[atlas1==val1]==val1) + np.sum(atlas2[atlas2==val2]==val2))
            
            #Store in ndarray
            Dice[int(i)][int(j)]=float(dice)

            print(f'Dice coefficient for {yname} {i} of {max_y}, {xname} {j} of {max_x} = {dice}')
            
            #Check for false Dice Coefficients and return what ROIs caused the issue
            if dice > 1 or dice < 0:
                raise ValueError(f"Dice coefficient is greater than 1 or less than 0 ({dice}) at atlas1: {val1}, atlas2: {val2}")

    #Save Dice map to csv file, comma delimited
    np.savetxt(f'{output_dir}/{png_name}.csv', Dice, delimiter=",")
    
    #Generate png of heatmap
    fig, ax = plt.subplots()
    #Create the heatmap:
    #cmap = color-scheme
    #norm = whether to make the colorbar logarithmic
    im = ax.imshow(Dice, cmap="gist_heat_r", norm=matplotlib.colors.LogNorm())

    #If there are more than 30 ROIs in an atlas, if so, then increase the step size for tick marks
    #on the x and y axes
    if len(labs1)<30:
        step1=1
    else:
        step1=floor(len(labs1)/30)

    if len(labs2)<30:
        step2=1
    else:
        step2=floor(len(labs2)/30)

    #Create tickmarks for axes
    ax.set_xticks(np.arange(0,len(labs2), step=step2))
    ax.set_yticks(np.arange(0,len(labs1), step=step1))

    #Add the label values to the corresponding tickmarks
    ax.set_xticklabels(labs2[0::step2])
    ax.set_yticklabels(labs1[0::step1])
    
    #Label x and y axes
    ax.set_ylabel(f'ROIs for {yname} atlas')
    ax.set_xlabel(f'ROIs for {xname} atlas')
    
    #Set the fontsize of the tickmark labels, rotate the x-axis labels 90 degrees to prevent overlap
    plt.setp(ax.get_xticklabels(), fontsize=6, rotation=90, ha="right", rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), fontsize=6)

    #Set title for the heatmap
    ax.set_title(f'{yname} vs {xname}')
    
    # Try to counteract the lopsided amount of ROIs between atlases
    # By changing the aspect ratio between the x and y axes of the graph
    aspect_ratio=len(labs2)/len(labs1)
    ax.set_aspect(aspect=aspect_ratio)

    plt.colorbar(im, aspect=30)
    fig.tight_layout()

    plt.show()
    #Save Dice map as a png file
    plt.savefig(f'{output_dir}/{png_name}.png', dpi=1000)


    return Dice, labs1, labs2
    print('Done')
    

Now the main segment of the progam is defined, the main function can be created. This function serves to organize the inputs and files provided by the user before feeding them into the dice_roi function.

In [4]:
def main():
    # All inputs listed below are required
    parser = ArgumentParser(
        description="Script to take already MNI-aligned atlas images and generate json file information."
    )
    parser.add_argument(
        "input_dir",
        help="Input directory",
        action="store",
    )
    parser.add_argument(
        "output_dir",
        help="""Path to directory you wish to store output
        heatmap.""",
        action="store",
    )
    parser.add_argument(
        "atlases",
        help="""List of names of the mri parcellations
        file you intend to process. Each atlas will be compared
        to eachother, so [a,b,c] will generate axb, axc, bxc.""",
        nargs="+",
    )
    

    # and ... begin!
    print("\nBeginning Dice ...")

    # ---------------- Parse CLI arguments ---------------- #
    result = parser.parse_args()
    input_dir = result.input_dir
    atlases = result.atlases
    output_dir = result.output_dir
    
    # Print the inputs for early issue detection
    print(f"input directory = {input_dir}")
    print(f"output directory = {output_dir}")
    print(f"atlases to be analyzed{atlases}")

    # Creation of output_dir if it doesn't exit
    if not os.path.isdir(output_dir):
        print(f"Making output directory: {output_dir}")
        os.makedirs(f"{output_dir}")

    # Itterate through every combination of the atlases provided (except duplicates)
    for i in range(len(atlases)):
        for j in range(len(atlases)):
            if j > i:
                Dice_matrix, ylabels, xlabels = dice_roi(input_dir,output_dir,atlases[i],atlases[j])

if __name__ == "__main__":
    main()


Beginning Dice ...
/Users/ross/Documents/neuroparc/atlases/label/Human
['AAL_space-MNI152NLin6_res-1x1x1.nii.gz', 'AICHAJoliot2015_space-MNI152NLin6_res-1x1x1.nii.gz']
/Users/ross/Documents/neuroparc/atlases
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 0 of 384 = 0.9557796459157021
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 1 of 384 = 1.0449002337964272e-06
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 2 of 384 = 6.270397297598107e-06
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 3 of 384 = 9.00926127709931e-05
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 4 of 384 = 0.0001332181558607206
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 5 of 384 = 2.055205251432591e-05
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 6 of 384 = 1.3932934863007031e-05
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 7 of 384 = 6.616521558803726e-06
Dice coefficient for AAL 0 of 116, AICHAJoliot2015 8 of 384 = 9.084544214320061e-05
Dice coefficient for AAL 0 of 116, AI

KeyboardInterrupt: 

## Outputs:
After running this script to completion, for every potential combination of atlases you should have the following in your output directory:
1. A file named Dice\_{atlas1}\_&\_{atlas2}\_res-{vox}x{vox}x{vox}.png, containing a heatmap of the dice coefficients for each ROI.
2. A file named Dice\_{atlas1}\_&\_{atlas2}\_res-{vox}x{vox}x{vox}.csv, containing the dice coefficient matrix information for the acccompanying png file.

## Common Errors:
- Issues may arise if atlases being compared have different voxel sizes, as the overlap measured by the dice coefficient may not be accurate
- If the atlases you are using do not contain 'space-MNI152NLin6_res-' or end in either '.nii' or '.nii.gz', issues will arise in the naming structure of the output files. Either rename your atlases or edit the first 5 lines of code of the third code cell.